# All libraries needed

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from itertools import product
import warnings
from pandas.errors import PerformanceWarning

# Suppress fragmentation warnings from pandas
warnings.filterwarnings("ignore", category=PerformanceWarning)

Below is the training code for the Baseline model, which requires a relatively long training time. The training results are saved as a CSV file in the working directory.


In [6]:
# load the dataset
real_volatility = pd.read_csv('real_volatility.csv')
print(real_volatility.head())
print(real_volatility.shape)

          0         1         2         3         4         5         6  \
0 -0.627425 -0.671709  0.052946 -0.311350 -0.927253  0.071091 -0.478141   
1  0.321562  0.384213  0.208457 -0.090778  0.353450  0.411101  0.212516   
2  0.095244  0.287023  0.212021  0.619140  0.533643 -0.071180  0.156178   
3 -0.163557  0.065380  0.033819 -0.043579 -0.026952 -0.261194 -0.432998   
4 -0.030986 -0.318869 -0.270497 -0.215789 -0.333496  0.211945  0.324132   

          7         8         9  ...       102       103       104       105  \
0 -0.003984 -0.013077  0.391148  ... -0.230983 -0.478164 -0.367516 -0.595283   
1  0.205651 -0.285932  0.155620  ... -0.185814  0.099969  0.047413  0.123790   
2  0.083400  0.244994  0.231762  ...  0.759816  0.100488  0.326271  0.623392   
3 -0.069229  0.146612 -0.491002  ... -0.260930  0.194976 -0.179241 -0.081816   
4 -0.268647  0.042330  0.179657  ... -0.483016 -0.074828 -0.167866 -0.448550   

        106       107       108       109       110       111  
0 -0

In [13]:
def metric(prediction, actual):
    prediction, actual = np.array(prediction), np.array(actual)
    eps = 1e-8
    Prediction = np.maximum(prediction, eps)
    Actual = np.maximum(actual, eps)
    RMSE = np.sqrt(np.mean((Prediction - Actual) ** 2))
    QLIKE = np.mean(Actual / Prediction - np.log(Actual / Prediction) - 1)
    RMPSE = np.sqrt(np.mean(np.square((actual - prediction) / actual))) * 100
    MAPE = np.mean(np.abs((actual - prediction) / actual)) * 100
    return RMSE, QLIKE, RMPSE, MAPE


In [24]:
# print summary for each model
def print_summary(MSE, QLIKE, MAPE, RMPSE, NAME):
    print(f'\nPerformance Summary for {NAME}')
    print(f"Average MSE: {np.mean(MSE):.6f}")
    print(f"Average QLIKE: {np.mean(QLIKE):.6f}")
    print(f"Average MAPE: {np.mean(MAPE):.6f}%")
    print(f"Average RMPSE: {np.mean(RMPSE):.6f}%")

# HAR-RV

In [31]:
# lists to store result
har_panel = pd.DataFrame()
RMSE_list = []
QLIKE_list = []
RMPSE_list = []
MAPE_list = []
# Dictionary to store best parameters for each stock
best_params_dict = {}

# Hyperparameter Tuning
lag_matrix = {
    'short': [1],
    'medium': [5, 7],
    'long': [22, 30]
}

lag_matrix = list(product(lag_matrix['short'], lag_matrix['medium'], lag_matrix['long']))

# go through each unique stock
for stock in real_volatility.columns:  
    stock_series = real_volatility[stock]

    # Initialise the best performance tracker
    final_MAPE = float('inf')
    final_Prediction = None
    final_Actual = None
    best_params = None  

    # Grid Search each combination
    for short, medium, long in lag_matrix:
        har_data = pd.DataFrame({'volatility': stock_series})      
        har_data['rv_short'] = stock_series.shift(short)
        har_data['rv_medium'] = stock_series.rolling(window=medium).mean().shift(1)
        har_data['rv_long'] = stock_series.rolling(window=long).mean().shift(1)
        har_data['trend'] = np.arange(len(stock_series))
        har_data = har_data.dropna()

        # Train-Validation-Test split
        number = len(har_data)
        train_size = int(number * 0.8)
        validation_size = int(number * 0.1)

        har_train = har_data.iloc[:train_size]
        har_validation = har_data.iloc[train_size:train_size + validation_size]
        har_test = har_data.iloc[train_size + validation_size:]

        feature_columns = ['rv_short', 'rv_medium', 'rv_long', 'trend']
        
        # Prepare the dataset 
        train_X = har_train[feature_columns]
        train_y = har_train['volatility']
        validation_X = har_validation[feature_columns]
        validation_y = har_validation['volatility']
        if train_X.empty or validation_X.empty:
            print(f"Skipping {stock} with lags (short:{short}, medium:{medium}, long:{long}) due to insufficient data.")
            continue

        # fit the linear model
        har_model = LinearRegression()
        har_model.fit(train_X, train_y)

        # evaluate on the validation set
        validation_Prediction = np.exp(har_model.predict(validation_X))
        validation_Actual = np.exp(validation_y)
        validation_RMSE, validation_QLIKE, validation_RMPSE, validation_MAPE = metric(validation_Prediction, validation_Actual)

        # check if this the best lag combination
        if validation_MAPE < final_MAPE:
            final_MAPE = validation_MAPE
            test_features = har_test[feature_columns]
            test_target = har_test['volatility']
            
            valid_index = test_features.dropna().index
            test_X = test_features.loc[valid_index]
            test_y = test_target.loc[valid_index]

            # evaluate on the test set
            final_Prediction = np.exp(har_model.predict(test_X))
            final_Actual = np.exp(test_y)
            
            # Store best parameters for this stock
            best_params = {'short': short, 'medium': medium, 'long': long}

    # Store Result
    if final_Prediction is not None:
        har_panel[stock] = pd.Series(final_Prediction, index=valid_index)

        RMSE, QLIKE, RMPSE, MAPE = metric(final_Prediction, final_Actual)
    
        MAPE_list.append(MAPE)
        RMSE_list.append(RMSE)
        QLIKE_list.append(QLIKE)
        RMPSE_list.append(RMPSE)
        
        # Store best parameters in the dictionary
        best_params_dict[stock] = best_params

print_summary(RMSE_list, QLIKE_list, MAPE_list, RMPSE_list, 'HAR-RV')

# Print best parameters for each stock
print("\nBest Hyperparameters for HAR-RV Model:")
print("-" * 50)
print(f"{'Stock':<15} {'Short Term':<10} {'Medium Term':<15} {'Long Term':<15}")
print("-" * 50)
for stock, params in best_params_dict.items():
    print(f"{stock:<15} {params['short']:<10} {params['medium']:<15} {params['long']:<15}")

/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/linear_m


Performance Summary for HAR-RV
Average MSE: 0.961730
Average QLIKE: 0.175619
Average MAPE: 44.274357%
Average RMPSE: 61.581100%

Best Hyperparameters for HAR-RV Model:
--------------------------------------------------
Stock           Short Term Medium Term     Long Term      
--------------------------------------------------
0               1          7               30             
1               1          5               30             
2               1          5               30             
3               1          5               30             
4               1          7               22             
5               1          5               22             
6               1          5               30             
7               1          7               30             
8               1          5               30             
9               1          5               22             
10              1          5               30             
11              1    

/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/linear_m

# Lag Model Linear Model

In [ ]:
# lists to store the value
lag_panel = pd.DataFrame()
RMSE_list = []
QLIKE_list = []
RMPSE_list = []
MAPE_list = []
# Dictionary to store best lag parameter for each stock
best_lag_dict = {}

# Hyperparameter tuni
# ng
Lag = [1, 2, 3, 4, 5]

# Go through each stock
for stock in real_volatility.columns:  
    stock_series = real_volatility[stock]

    # Initialise for the best performance tracker
    final_MAPE = float('inf')
    final_Prediction = None
    final_Actual = None
    final_index = None
    best_lag = None  

    # go through each lag threshold
    for lag in Lag:
        
        # feature set
        lag_data = pd.DataFrame({
            'volatility': stock_series,
            'lag': stock_series.shift(lag),
            'trend': np.arange(len(stock_series))})
        lag_data = lag_data.dropna()

        # Train-Validation-Test split
        number = len(lag_data)
        train_size = int(number * 0.8)
        validation_size = int(number * 0.1)

        lag_train = lag_data.iloc[:train_size]
        lag_Validation = lag_data.iloc[train_size:train_size + validation_size]
        lag_test = lag_data.iloc[train_size + validation_size:]

        # Prepare the dataset
        feature_columns = ['lag', 'trend']
        train_X = lag_train[feature_columns]
        train_y = lag_train['volatility']
        validation_X = lag_Validation[feature_columns]
        validation_y = lag_Validation['volatility']
        test_X = lag_test[feature_columns]
        test_y = lag_test['volatility']
        test_X = test_X.dropna()
        test_y = test_y.loc[test_X.index]

        # fit the model
        lag_model = LinearRegression()
        lag_model.fit(train_X, train_y)

        # evaluate on the validation set
        validation_Prediction = np.exp(lag_model.predict(validation_X))
        validation_Actual = np.exp(validation_y)
        validation_RMSE, validation_QLIKE, validation_RMPSE, validation_MAPE = metric(validation_Prediction, validation_Actual)

        # Check is it the best performance
        if validation_MAPE < final_MAPE:
            final_MAPE = validation_MAPE
            final_Prediction = np.exp(lag_model.predict(test_X))
            final_Actual = np.exp(test_y)
            final_index = test_X.index
            best_lag = lag  # Store the best lag

    # store the result
    if final_Prediction is not None:
        lag_panel[stock] = pd.Series(final_Prediction, index=final_index)
        
        RMSE, QLIKE, RMPSE, MAPE = metric(final_Prediction, final_Actual)
    
        MAPE_list.append(MAPE)
        RMSE_list.append(RMSE)
        QLIKE_list.append(QLIKE)
        RMPSE_list.append(RMPSE)

        # Store best lag in the dictionary
        best_lag_dict[stock] = best_lag
        
# print the overall result
print_summary(RMSE_list, QLIKE_list, MAPE_list, RMPSE_list, 'Lag Model')
        
# Print best lag parameters for each stock
print("\nBest Lag Parameters for Lag Model:")
print("-" * 40)
print(f"{'Stock':<15} {'Best Lag':<10}")
print("-" * 40)
for stock, lag in best_lag_dict.items():
    print(f"{stock:<15} {lag:<10}")


Performance Summary for Lag Model
Average MSE: 1.016574
Average QLIKE: 0.215881
Average MAPE: 50.660738%
Average RMPSE: 76.913119%

Best Lag Parameters for Lag Model:
----------------------------------------
Stock           Best Lag  
----------------------------------------
0               1         
1               1         
2               1         
3               1         
4               1         
5               1         
6               1         
7               1         
8               1         
9               1         
10              1         
11              1         
12              1         
13              1         
14              1         
15              1         
16              1         
17              1         
18              1         
19              1         
20              1         
21              1         
22              1         
23              1         
24              1         
25              1         
26              1    

# PCA model

In [28]:
# List to store value
pca_linear_panel = pd.DataFrame()
RMSE_list = []
QLIKE_list = []
RMPSE_list = []
MAPE_list = []
# Dictionary to store best variance threshold for each stock
best_variance_dict = {}

# Hyperparameters tuning
variance_threshold = [0.9, 0.93, 0.95, 0.97, 0.99]

# Go through each individual stock
for stock in real_volatility.columns:  
    stock_series = real_volatility[stock]

    # feature set
    pca_data = pd.DataFrame({
        'volatility': stock_series,
        'trend': np.arange(len(stock_series)),
        'lag1': stock_series.shift(1),
        'lag2': stock_series.shift(2),
        'lag3': stock_series.shift(3),
        'moving_avg_3': stock_series.rolling(window=3).mean().shift(1),
        'moving_avg_5': stock_series.rolling(window=5).mean().shift(1),
        'moving_avg_10': stock_series.rolling(window=10).mean().shift(1),
        'std_5': stock_series.rolling(window=5).std().shift(1),
        'std_10': stock_series.rolling(window=10).std().shift(1)})

    pca_data = pca_data.dropna()

    # Train-Validation-Test split
    number = len(pca_data)
    train_size = int(number * 0.8)
    validation_size = int(number * 0.1)

    pca_train = pca_data.iloc[:train_size]
    pca_validation = pca_data.iloc[train_size:train_size + validation_size]
    pca_test = pca_data.iloc[train_size + validation_size:]

    # Prepare the dataset 
    feature_columns = ['trend', 'lag1', 'lag2', 'lag3', 'moving_avg_3', 'moving_avg_5', 'moving_avg_10', 'std_5', 'std_10']
    
    train_X = pca_train[feature_columns]
    train_y = pca_train['volatility']
    validation_X = pca_validation[feature_columns]
    validation_y = pca_validation['volatility']
    test_X = pca_test[feature_columns]
    test_y = pca_test['volatility']

    # Scaling
    scaler = StandardScaler()
    train_X_scaled = scaler.fit_transform(train_X)
    validation_X_scaled = scaler.transform(validation_X)
    test_X_scaled = scaler.transform(test_X)

    # Initialise for the best performance tracker
    final_MAPE = float('inf')
    final_Prediction = None
    final_Actual = None
    final_variance = None

    # Go through each variance threshold
    for variance in variance_threshold:  
        pca = PCA(n_components=variance)     
        train_pca = pca.fit_transform(train_X_scaled)
        validation_pca = pca.transform(validation_X_scaled)

        pca_linear_model = LinearRegression()
        pca_linear_model.fit(train_pca, train_y)

        validation_Prediction = np.exp(pca_linear_model.predict(validation_pca))
        validation_Actual = np.exp(validation_y)
        validation_RMSE, validation_QLIKE, validation_RMPSE, validation_MAPE = metric(validation_Prediction, validation_Actual)

        # check if this is the best variance 
        if validation_MAPE < final_MAPE:
            final_MAPE = validation_MAPE
            final_variance = variance
            final_model = pca_linear_model
            final_pca = pca
        
    # evaluate on test set    
    test_pca = final_pca.transform(test_X_scaled)
    test_prediction = final_model.predict(test_pca)
    final_Prediction = np.exp(test_prediction)
    final_Actual = np.exp(test_y)
    pca_linear_panel[stock] = pd.Series(final_Prediction, index=pca_test.index)

    RMSE, QLIKE, RMPSE, MAPE = metric(final_Prediction, final_Actual)
    
    MAPE_list.append(MAPE)
    RMSE_list.append(RMSE)
    QLIKE_list.append(QLIKE)
    RMPSE_list.append(RMPSE)
    
    # Store best variance in the dictionary
    best_variance_dict[stock] = final_variance

# print final evaluation 
print_summary(RMSE_list, QLIKE_list, MAPE_list, RMPSE_list, 'PCA Model')

# Print best variance parameters for each stock
print("\nBest Variance Thresholds for PCA Model:")
print("-" * 40)
print(f"{'Stock':<15} {'Variance Threshold':<20}")
print("-" * 40)
for stock, variance in best_variance_dict.items():
    print(f"{stock:<15} {variance:<20}")

/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encountered in matmul
  X_transformed = X @ self.components_.T
/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/ruohaitao/Library/Python/3.12/lib/python/site-


Performance Summary for PCA Model
Average MSE: 0.957018
Average QLIKE: 0.179356
Average MAPE: 39.664900%
Average RMPSE: 53.604946%

Best Variance Thresholds for PCA Model:
----------------------------------------
Stock           Variance Threshold  
----------------------------------------
0               0.99                
1               0.99                
2               0.93                
3               0.99                
4               0.99                
5               0.99                
6               0.99                
7               0.93                
8               0.99                
9               0.99                
10              0.99                
11              0.9                 
12              0.93                
13              0.93                
14              0.99                
15              0.99                
16              0.99                
17              0.99                
18              0.93                
19   

/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encountered in matmul
  X_transformed = X @ self.components_.T
/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/ruohaitao/Library/Python/3.12/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/ruohaitao/Library/Python/3.12/lib/python/site-

# Random Forest

In [ ]:
rf_panel = pd.DataFrame()
RMSE_list = []
QLIKE_list = []
RMPSE_list = []
MAPE_list = []
# Dictionary to store best n_estimators for each stock
best_estimators_dict = {}

n_estimators = [200, 300, 500]

for stock in real_volatility.columns:  
    stock_series = real_volatility[stock]

    rf_data = pd.DataFrame({
        'volatility': stock_series,
        'trend': np.arange(len(stock_series)),
        'lag1': stock_series.shift(1),
        'lag2': stock_series.shift(2),
        'lag3': stock_series.shift(3),
        'moving_avg_3': stock_series.rolling(window=3).mean().shift(1),
        'moving_avg_5': stock_series.rolling(window=5).mean().shift(1),
        'moving_avg_10': stock_series.rolling(window=10).mean().shift(1),
        'std_5': stock_series.rolling(window=5).std().shift(1),
        'std_10': stock_series.rolling(window=10).std().shift(1),
        'max_5': stock_series.rolling(window=5).max().shift(1),
        'min_5': stock_series.rolling(window=5).min().shift(1)})

    rf_data = rf_data.dropna()

    number = len(rf_data)
    train_size = int(number * 0.8)
    validation_size = int(number * 0.1)

    rf_train = rf_data.iloc[:train_size]
    rf_validation = rf_data.iloc[train_size:train_size + validation_size]
    rf_test = rf_data.iloc[train_size + validation_size:]

    feature_columns = ['trend', 'lag1', 'lag2', 'lag3', 'moving_avg_3', 'moving_avg_5', 'moving_avg_10', 'std_5', 'std_10', 'max_5', 'min_5']
    
    train_X = rf_train[feature_columns]
    train_y = rf_train['volatility']
    validation_X = rf_validation[feature_columns]
    validation_y = rf_validation['volatility']
    test_X = rf_test[feature_columns]
    test_y = rf_test['volatility']

    final_MAPE = float('inf')
    final_Prediction = None
    final_Actual = None
    best_n = None  # Track best n_estimators

    for n in n_estimators:
        rf_model = RandomForestRegressor(n_estimators=n, random_state=42, max_depth=10)
        rf_model.fit(train_X, train_y)
        validation_Prediction = np.exp(rf_model.predict(validation_X))
        validation_Actual = np.exp(validation_y)
        validation_RMSE, validation_QLIKE, validation_RMPSE, validation_MAPE = metric(validation_Prediction, validation_Actual)

        if validation_MAPE < final_MAPE:
            final_MAPE = validation_MAPE
            final_Prediction = np.exp(rf_model.predict(test_X))
            final_Actual = np.exp(test_y)
            best_n = n  # Store best n_estimators

    rf_panel[stock] = pd.Series(final_Prediction, index=rf_test.index)

    RMSE, QLIKE, RMPSE, MAPE = metric(final_Prediction, final_Actual)
    
    MAPE_list.append(MAPE)
    RMSE_list.append(RMSE)
    QLIKE_list.append(QLIKE)
    RMPSE_list.append(RMPSE)
    
    # Store best parameter in the dictionary
    best_estimators_dict[stock] = best_n

# Print Result
print_summary(RMSE_list, QLIKE_list, MAPE_list, RMPSE_list, 'Random Forest')
# Print best parameters for each stock
print("\nBest n_estimators for Random Forest Model:")
print("-" * 40)
print(f"{'Stock':<15} {'n_estimators':<15}")
print("-" * 40)
for stock, n_est in best_estimators_dict.items():
    print(f"{stock:<15} {n_est:<15}")

# Gradient Boosting

In [ ]:
gb_panel = pd.DataFrame()
QLIKE_list = []
MSE_list = []
DA_list
MAPE_list = []
# Dictionary to store best learning rate for each stock
best_lr_dict = {}

learning_rate = [0.01, 0.05, 0.1]

for stock in real_volatility.columns:  
    stock_series = real_volatility[stock]

    gb_data = pd.DataFrame({
        'volatility': stock_series,
        'trend': np.arange(len(stock_series)),
        'lag1': stock_series.shift(1),
        'lag2': stock_series.shift(2),
        'lag3': stock_series.shift(3),
        'moving_avg_3': stock_series.rolling(window=3).mean().shift(1),
        'moving_avg_5': stock_series.rolling(window=5).mean().shift(1),
        'moving_avg_10': stock_series.rolling(window=10).mean().shift(1),
        'std_5': stock_series.rolling(window=5).std().shift(1),
        'std_10': stock_series.rolling(window=10).std().shift(1),
        'max_5': stock_series.rolling(window=5).max().shift(1),
        'min_5': stock_series.rolling(window=5).min().shift(1)})

    gb_data = gb_data.dropna()

    number = len(gb_data)
    train_size = int(number * 0.8)
    validation_size = int(number * 0.1)

    gb_train = gb_data.iloc[:train_size]
    gb_validation = gb_data.iloc[train_size:train_size + validation_size]
    gb_test = gb_data.iloc[train_size + validation_size:]

    feature_columns = ['trend', 'lag1', 'lag2', 'lag3', 'moving_avg_3', 'moving_avg_5', 'moving_avg_10', 'std_5', 'std_10', 'max_5', 'min_5']
    
    train_X = gb_train[feature_columns]
    train_y = gb_train['volatility']
    validation_X = gb_validation[feature_columns]
    validation_y = gb_validation['volatility']
    test_X = gb_test[feature_columns]
    test_y = gb_test['volatility']

    final_MAPE = float('inf')
    final_Prediction = None
    final_Actual = None
    best_lr = None  # Track best learning rate

    for learning in learning_rate:
        gb_model = GradientBoostingRegressor(n_estimators=500, learning_rate=learning, max_depth=6, random_state=42)
        gb_model.fit(train_X, train_y)
        validation_Prediction = np.exp(gb_model.predict(validation_X))
        validation_Actual = np.exp(validation_y)
        validation_QLIKE, validation_MAPE, validation_MSE, validation_DA = metric(validation_Prediction, validation_Actual)

        if validation_MAPE < final_MAPE:
            final_MAPE = validation_MAPE
            final_Prediction = np.exp(gb_model.predict(test_X))
            final_Actual = np.exp(test_y)
            best_lr = learning  # Store best learning rate

    gb_panel[stock] = pd.Series(final_Prediction, index=gb_test.index)

    QLIKE, MAPE, MSE, DA = metric(final_Prediction, final_Actual)
    
    MAPE_list.append(MAPE)
    MSE_list.append(MSE)
    QLIKE_list.append(QLIKE)
    DA_list.append(DA)
    
    # Store best learning rate in the dictionary
    best_lr_dict[stock] = best_lr

# Print Results
print_summary(MSE_list, QLIKE_list, MAPE_list, DA_list, 'Gradient Boosting')

# Print best learning rates for each stock
print("\nBest Learning Rates for Gradient Boosting Model:")
print("-" * 40)
print(f"{'Stock':<15} {'Learning Rate':<15}")
print("-" * 40)
for stock, lr in best_lr_dict.items():
    print(f"{stock:<15} {lr:<15}")

# Linear Regression

In [ ]:
# List to store value
linear_panel = pd.DataFrame()
RMSE_list = []
QLIKE_list = []
RMPSE_list = []
MAPE_list = []
# Dictionary to store best moving average for each stock
best_ma_dict = {}

# Hyperparameters tuning
moving_average = [3, 5, 7, 10]

# go through each stock
for stock in real_volatility.columns:  
    stock_series = real_volatility[stock]

    # initialise for the best performance tracker
    final_MAPE = float('inf')
    final_Prediction = None
    final_Actual = None
    best_ma = None  # Track best moving average window

    # go through each moving average threshold
    for moving in moving_average: 

        # feature sets
        linear_data = pd.DataFrame({
            'volatility': stock_series,
            'trend': np.arange(len(stock_series)),
            'lag1': stock_series.shift(1),
            'lag2': stock_series.shift(2),
            'moving_avg': stock_series.rolling(window=moving).mean().shift(1)})
        linear_data = linear_data.dropna()

        # Train-Validation-Test split
        number = len(linear_data)
        train_size = int(number * 0.8)
        validation_size = int(number * 0.1)

        linear_train = linear_data.iloc[:train_size]
        linear_validation = linear_data.iloc[train_size:train_size + validation_size]
        linear_test = linear_data.iloc[train_size + validation_size:]

        # Prepare the dataset 
        feature_columns = ['trend', 'lag1', 'lag2', 'moving_avg']
        train_X = linear_train[feature_columns]
        train_y = linear_train['volatility']
        validation_X = linear_validation[feature_columns]
        validation_y = linear_validation['volatility']


        # fit on train set
        linear_model = LinearRegression()
        linear_model.fit(train_X, train_y)

        # evaluate on the validation set
        validation_Prediction = np.exp(linear_model.predict(validation_X))
        validation_Actual = np.exp(validation_y)
        validation_RMSE, validation_QLIKE, validation_RMPSE, validation_MAPE = metric(validation_Prediction, validation_Actual)

        # check is this the best
        if validation_MAPE < final_MAPE:
            final_MAPE = validation_MAPE
            test_data = linear_test.dropna(subset=feature_columns + ['volatility'])
            test_X = test_data[feature_columns]
            test_y = test_data['volatility'] 
            final_Prediction = np.exp(linear_model.predict(test_X))
            final_Actual = np.exp(test_y)
            best_ma = moving  # Store the best moving average
            

    # store the value for each stock
    if final_Prediction is not None:
        linear_panel[stock] = pd.Series(final_Prediction, index=test_X.index)

        RMSE, QLIKE, RMPSE, MAPE = metric(final_Prediction, final_Actual)
    
        MAPE_list.append(MAPE)
        RMSE_list.append(RMSE)
        QLIKE_list.append(QLIKE)
        RMPSE_list.append(RMPSE)
        
        # Store best moving average window in the dictionary
        best_ma_dict[stock] = best_ma

# Print summary
print_summary(RMSE_list, QLIKE_list, MAPE_list, RMPSE_list, 'Linear Regression')

# Print best moving average windows for each stock
print("\nBest Moving Average Windows for Linear Regression Model:")
print("-" * 40)
print(f"{'Stock':<15} {'Moving Average':<15}")
print("-" * 40)
for stock, ma in best_ma_dict.items():
    print(f"{stock:<15} {ma:<15}")

In [23]:
import os
from pathlib import Path

# Use current directory as save path
save_path = Path('.')

# All model panels
models_panels = {
    'LAG': lag_panel,
    'HAR_RV': har_panel, 
    'Linear': linear_panel,
    'PCA_Linear': pca_linear_panel,
    'Random_Forest': rf_panel,
    'Gradient_Boosting': gb_panel
}

# 1. Save all model panels to CSV
print("Saving model panels to current directory...")
for model_name, panel in models_panels.items():
    filename = f"{model_name}_predictions.csv"
    filepath = save_path / filename
    panel.to_csv(filepath)
    print(f"Saved: {filename}")



# 2. Calculate metrics for all models and generate table
def calculate_metrics(predictions, actuals):
    # Use the metric function defined at the top of the file
    # which already returns RMSE, QLIKE, RMPSE, MAPE
    return metric(predictions, actuals)

# Get common index and columns
common_index = None
common_stocks = None
for model_name, panel in models_panels.items():
    if common_index is None:
        common_index = panel.index
        common_stocks = panel.columns
    else:
        common_index = common_index.intersection(panel.index)
        common_stocks = common_stocks.intersection(panel.columns)

# Align actual values
aligned_real = real_volatility.loc[common_index, common_stocks]

# Calculate metrics for each model
metrics_data = []
for model_name, panel in models_panels.items():
    aligned_pred = panel.loc[common_index, common_stocks]
    
    pred_values = aligned_pred.values.flatten()
    real_values = aligned_real.values.flatten()
    
    # Remove NaN values
    mask = ~(np.isnan(pred_values) | np.isnan(real_values))
    pred_clean = pred_values[mask]
    real_clean = real_values[mask]
    
    rmse, qlike, rmpse_val, mape_val = calculate_metrics(pred_clean, real_clean)
    metrics_data.append({
        'Model': model_name,
        'RMSE': rmse,
        'QLIKE': qlike,
        'RMPSE': rmpse_val,
        'MAPE': mape_val
    })

# Create metrics table
metrics_df = pd.DataFrame(metrics_data)
metrics_df = metrics_df.round(6)

# Save metrics table
metrics_df.to_csv(save_path / "model_metrics_summary.csv", index=False)

# Display table
print("\nModel Performance Metrics:")
print("="*60)
print(metrics_df.to_string(index=False))

Saving model panels to current directory...
Saved: LAG_predictions.csv
Saved: HAR_RV_predictions.csv
Saved: Linear_predictions.csv
Saved: PCA_Linear_predictions.csv
Saved: Random_Forest_predictions.csv
Saved: Gradient_Boosting_predictions.csv

Model Performance Metrics:
            Model     RMSE    QLIKE        RMPSE         MAPE
              LAG 0.941789 8.849587 4.632589e+09 3.957171e+17
           HAR_RV 0.977866 8.851011 4.525824e+09 3.854161e+17
           Linear 0.967299 8.843727 4.531514e+09 3.883879e+17
       PCA_Linear 0.917355 8.804401 4.244223e+09 3.398432e+17
    Random_Forest 0.955538 8.832101 4.420879e+09 3.702024e+17
Gradient_Boosting 1.043312 8.848217 4.519398e+09 4.077799e+17
